In [5]:
import RNA
import numpy as np

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

import subprocess
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG, display
from collections import Counter
from collections import defaultdict

import difflib
import sys
import os
import random
import string
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# from helper import print_moves

import pandas as pd

sys.path.append('../')
from pretty_print_path import print_moves
import findpath_librna
import findpath


In [6]:
input_file = "dataset_102_train.csv"

dataframe = pd.read_csv(input_file)

dataframe['target'] = np.where(dataframe["3"]==1, 1, 0)

dataframe = dataframe.drop(labels="0", axis=1)
dataframe = dataframe.drop(labels="1", axis=1)
dataframe = dataframe.drop(labels="2", axis=1)
dataframe = dataframe.drop(labels="9", axis=1)
dataframe = dataframe.drop(labels="3", axis=1)
dataframe

,Unnamed: 0,4,5,6,7,8,target
0,0,0.363636,0.285714,0.142857,1,0.000000,1
1,1,0.181818,0.333333,0.142857,1,0.034483,1
2,2,0.818182,0.238095,0.142857,1,0.034483,0
3,3,0.954545,0.047619,0.142857,1,0.103448,0
4,4,0.636364,0.333333,0.142857,1,0.206897,1
...,...,...,...,...,...,...,...
49492,49492,0.666667,1.000000,0.000000,0,1.000000,1
49493,49493,0.200000,1.000000,1.000000,1,0.000000,1
49494,49494,0.800000,1.000000,1.000000,0,0.760000,1
49495,49495,0.400000,1.000000,1.000000,0,0.920000,1


In [7]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

31677 train examples
7920 validation examples
9900 test examples


In [8]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## Understand the input pipeline
# Now that we have created the input pipeline, let's call it to see the format of the data it returns. We have used a small batch size to keep the output readable.

for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['4'])
  print('A batch of targets:', label_batch )



Every feature: ['Unnamed: 0', '4', '5', '6', '7', '8']
A batch of ages: tf.Tensor([0.80769231 0.08333333 0.2173913  0.65217391 0.375     ], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([0 1 1 0 1], shape=(5,), dtype=int64)


In [9]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

# numeric columns
photo_count = feature_column.numeric_column('4')
demo(photo_count)

# column 7 is a catigorical column
animal_type = feature_column.categorical_column_with_vocabulary_list(
      '7', [0, 1])
animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)

[[0.09090909]
 [0.3       ]
 [0.75      ]
 [0.21052632]
 [0.11111111]]
[[1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


## Choose which columns to use
We have seen how to use several types of feature columns. Now we will use them to train a model. The goal of this tutorial is to show you the complete code (e.g. mechanics) needed to work with feature columns. We have selected a few columns to train our model below arbitrarily.

Key point: If your aim is to build an accurate model, try a larger dataset of your own, and think carefully about which features are the most meaningful to include, and how they should be represented.

In [13]:
feature_columns = []

# numeric cols
for header in ['4', '5', '6', '8']:
  feature_columns.append(feature_column.numeric_column(header))

# categoriacal indicator_columns
indicator_column_names = ['7']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [11]:
# only use lowest energy as indicator

feature_columns = []

# numeric cols
for header in ['8']:
  feature_columns.append(feature_column.numeric_column(header))

In [14]:
feature_columns

[NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='7', vocabulary_list=(1, 0), dtype=tf.int64, default_value=-1, num_oov_buckets=0))]

### Create a feature layer
Now that we have defined our feature columns, we will use a [DenseFeatures](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/DenseFeatures) layer to input them to our Keras model.

Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.

In [15]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [16]:
epochs = 500

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=epochs)

Epoch 1/500
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
971/990 [============================>.] - ETA: 0s - loss: 0.5485 - accuracy: 0.7248WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Unnamed: 0': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=int64>, '4': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, '5': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, '6': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=float64>, '7': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=int64>, '8': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=float64>}
Consider rewriting this model with the Functional API.
990/990 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7243 - val_loss: 0.5388 - val_accuracy: 0.7439
Epoch 2/500
990/990 [==============================] - 2s 2ms/step - loss: 0.5360 - acc

In [17]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

310/310 [==============================] - 0s 951us/step - loss: 0.5596 - accuracy: 0.7365
Accuracy 0.7364646196365356


In [ ]:
# 31/31 [==============================] - 0s 1ms/step - loss: 0.6488 - accuracy: 0.7335
# Accuracy 0.7335359454154968

# 31/31 [==============================] - 0s 862us/step - loss: 0.5798 - accuracy: 0.6950
# Accuracy 0.695035457611084

In [23]:
# Save the entire model as a SavedModel.
# !mkdir -p saved_model
# model.save('saved_model/my_model')

from tensorflow.keras.models import Sequential, save_model, load_model

# filepath = './saved_model'
# save_model(model, filepath)

# Morgan & Higgs: Findpath with SW 1

In [116]:
input_file = "dataset_100.csv"
df = pd.read_csv(input_file)

sequence, s1, s2 = df.loc[0]


search_width_multiplier = 4
fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=True)
result = fp.get_en()/100.0
path = fp.get_path()

s = s1
pt2 = list(RNA.ptable(s2))
fc = RNA.fold_compound(sequence)


def find_moves(s_ptable, t_ptable):
    """
    generator function, yields possible structures 1 move away
    from the original structure by finding fitting i and j with
    RNA pair and loop tables
    s_ptable: current ptable
    t_ptable: s2 end ptable
    """
    # loop table
    ls = RNA.loopidx_from_ptable(s_ptable)
    for i in range(len(s_ptable)):
        if i == 0:
            continue
        if s_ptable[i] == 0 and t_ptable[i] > i:
            j = t_ptable[i]
            # found j has to be empty and currently on the same loop as i
            if s_ptable[j] == 0 and ls[i] == ls[j]:
                yield i, j
        # test for bp removal: i has to be paired with a different j in s2
        j = s_ptable[i]
        # dont remove things which are present in s2
        if s_ptable[i] > i and s_ptable[i] != s_ptable[j] and\
                s_ptable[i] != t_ptable[i] and s_ptable[j] != t_ptable[j]:
            yield -i, -j


def fp_call(sequence, s1, s2, search_width_multiplier = 20):    
    fp = findpath.findpath_single(sequence, s1, s2, search_width_multiplier=search_width_multiplier, mp=True)
    result = fp.get_en()/100.0
    path = fp.get_path()
    # return result, path
    return result, path


def ij_distance(last_move, this_move, ij_moves):
    # how far is the last move away from the current move.
    # it is likely, that the next move is close to the last one
    # there are better distance metrices probably...

    # ij move list is supposed to be sorted, find indices
    
    ijmoves = ij_moves + [last_move]
    ijmoves.sort(key=lambda x: (x[0], x[1]))

    pos_old = ijmoves.index(last_move)
    pos_new = ijmoves.index(this_move)

    distance = abs(pos_old-pos_new)/len(ijmoves)
    
    
    # moves left in vicinity out of total moves
    
    thisi, thisj = this_move
    lasti, lastj = last_move
    thisclose = 0
    lastclose = 0

    for i, j in ij_moves:
        if (abs(i-thisi) < 5) and (abs(j-thisj) < 5):
            thisclose += 1
        if (abs(i-lasti) < 5) and (abs(j-lastj) < 5):
            lastclose += 1
    

    thisclose /= len(ij_moves)
    lastclose /= len(ij_moves)

    # print ("thisclose", thisclose, lastclose, distance)
    return distance, thisclose, lastclose

    print (distance)

# sample call
ij_moves = [(3, 62), (4, 61), (6, 60), (7, 59), (9, 57), (10, 56), (11, 55), (12, 54), (15, 52), (16, 51), (17, 50), (18, 39), (19, 38), (20, 36), (21, 35), (22, 34), (23, 33), (24, 32), (42, 49), (43, 48), (64, 99)]
last_move = (2, 63)
this_move = (6, 60)
ij_distance(last_move, this_move, ij_moves)

# 

def config_distance(pt, move):
    """
    are we extending / removing the outside/inside layer of a loop or adding something in the middle?
    """
    i = move[0]
    j = move[1]
    points = 0

    # if we're extending from outside to inside, the position i+1 and j-1 should be ideally unpaired
    # inside to outside: i-1 and j+1 should be ideally unpaired

    if i>0:
        # print ("add") 
        # outside/inside paired?        
        if j+1 < pt[0] and i-1 > 0: # outside - boundary check
            if pt[i-1] == j+1:
                points += 1
        if pt[i+1] == j-1:
            points += 1
    if i<0:
        # print ("del")
        i, j = -i, -j
        # outside/inside paired?
        if j+1 < pt[0] and i-1 > 0: # outside - boundary check
            if pt[i-1] == j+1:
                points += 1
        if pt[i+1] == j-1:
            points += 1
        elif pt[i+1] == 0 and pt[j-1] == 0:
            pass

    if points == 2:
        points = 0
    return points


s = s1
lasts = s
lasti = None
lastj = None

for e, (a,b, en) in enumerate(path):
    if (a,b) == (0,0):
        continue  

    # check where we can go, compare with our best move. 
    pt = list(RNA.ptable(s))

    # check available moves, save them, sort them    
    avail_moves = []
    ij_moves = []
    found_pos = None

    for pos, (i,j) in enumerate(find_moves(pt, pt2)):    
        next_en = fc.eval_move_pt(pt, i, j)
        # mark where we found our move
        found = (i,j) == (a,b)
        avail_moves.append((i, j, next_en, found))
        ij_moves.append((abs(i),abs(j)))

    # sort moves independent of delete insert moves

    ij_moves.sort(key=lambda x: (x[0], x[1]))


    avail_moves.sort(key=lambda x: x[2])
    found_list = [x[3] for x in avail_moves]
    en_list = np.array([[x[2] for x in avail_moves]])
    en_list_scaled = min_max_scaler.fit_transform(en_list.T).T[0]
    

    # find where our move is after sorting
    found_pos = found_list.index(True)
    rel_pos = found_pos * 1.0 / len(found_list)

    # print (e, a,b, 'found at pos:', found_pos, 'of', len(avail_moves), ':',  1-rel_pos)
    # print (avail_moves, a, b)

    # if s != lasts:
        # print ("---") 
        # print (s)
        # print ("---") 

    # for every move we take we have to run a new findpath, see if this move will yield the ideal result
    
    for pos, (i,j, en, found) in enumerate(avail_moves):
        if i > 0:
            snew = s[:i-1] + "(" + s[i:j-1] + ")" + s[j:]
        if i < 0:
            snew = s[:-i-1] + "." + s[-i:-j-1] + "." + s[-j:]
        ptnew = list(RNA.ptable(snew))

        result_new, path = fp_call(sequence, snew, s2)

        if result_new <= result:
            pos_result = 1
        else:
            pos_result = 0

        if found: found = "<-- taken"
        else: found = ""

        this_move = (abs(i), abs(j))
        last_move = (lasti, lastj)

        if lasti:
            # print (this_move, last_move, ij_moves)
            ijd, thisclose, lastclose = ij_distance(last_move, this_move, ij_moves)
        else:
            ijd, thisclose, lastclose = 0, 0, 0

        cd = config_distance(pt, this_move)

        if lasti:
            print (f' {snew}, {i}, {j}, {result_new}/{pos_result}: {ijd:2.2f}, {thisclose:2.2f}, {lastclose:2.2f}, {cd}, {en}, {en_list_scaled[pos]:2.2f} {found}')
            print (f' {1}, {ijd:2.2f}, {thisclose:2.2f}, {lastclose:2.2f}, {cd}, {en_list_scaled[pos]:2.2f}, 1')

    # if e==63:
    #     print (avail_moves)

    # update s for the next iteration
    
    lasts = s
    lasti = abs(a)
    lastj = abs(b)
    if a > 0:
        s = s[:a-1] + "(" + s[a:b-1] + ")" + s[b:]
    if a < 0:
        s = s[:-a-1] + "." + s[-a:-b-1] + "." + s[-b:]

    if e>1: 
        break



 ..((.((.(((...((((((((((.......))))).))..........)))..))).)))).(((.((((((..((((.....))))..)))))))))., -12, -54, -14.2/1: 0.36, 0.29, 0.14, 1, 90, 0.00 <-- taken
 1, 0.36, 0.29, 0.14, 1, 0.00, 1
 ..((.(..((((..((((((((((.......))))).))..........))).))))..))).(((.((((((..((((.....))))..)))))))))., -7, -59, -14.2/1: 0.18, 0.33, 0.14, 1, 110, 0.03 
 1, 0.18, 0.33, 0.14, 1, 0.03, 1
 ..((.((.((((..(((((((((.........)))).))..........))).)))).)))).(((.((((((..((((.....))))..)))))))))., -24, -32, -13.3/0: 0.82, 0.24, 0.14, 1, 110, 0.03 
 1, 0.82, 0.24, 0.14, 1, 0.03, 1
 ..((.((.((((..((((((((((.......))))).))..........))).)))).))))..((.((((((..((((.....))))..)))))))).., -64, -99, -12.7/0: 0.95, 0.05, 0.14, 1, 150, 0.10 
 1, 0.95, 0.05, 0.14, 1, 0.10, 1
 ..((.((.((((..(((((.((((.......))))..))..........))).)))).)))).(((.((((((..((((.....))))..)))))))))., -20, -36, -14.2/1: 0.64, 0.33, 0.14, 1, 210, 0.21 
 1, 0.64, 0.33, 0.14, 1, 0.21, 1
 ..((.((.((((..((((.(((((.......)))))..)..........))).))))

In [46]:
# A few random samples


# Generate predictions for samples
predictions = model.predict(test_ds)

predictions
sample_test
# test_ds

,Unnamed: 0,4,5,6,7,8,target
39424,39424,0.227273,0.380952,0.095238,0,0.691358,0
44489,44489,0.166667,0.130435,0.086957,1,0.750000,0
30368,30368,0.769231,0.160000,0.040000,0,0.485294,0
38959,38959,0.100000,0.210526,0.157895,0,0.846154,0
24553,24553,0.333333,0.285714,0.071429,1,0.705128,0
24716,24716,0.230769,0.250000,0.000000,0,0.937107,0
12419,12419,0.500000,0.714286,0.142857,0,0.758621,0
4130,4130,0.153846,0.250000,0.250000,1,0.716981,0
12647,12647,0.583333,0.454545,0.090909,1,0.547270,0
32613,32613,0.533333,0.428571,0.000000,0,0.856459,0


In [122]:
# dataframe['target']


sample_test = test.iloc[0:1]
# print (sample_test)
# sample_test.loc[0]   = 0, 0, 0.5, 0,0,1,1 # last column = true value, does nothing

sample_test.loc[39424] = 12, 0.36, 0.29, 0.14, 1, 0.00, 1 # should be 1
sample_test.loc[39424] =  1, 0.82, 0.24, 0.14, 1, 0.03, 1
sample_test.loc[39424] =  1, 0.23, 0.29, 0.14, 1, 0.38, 1




print (sample_test)
# sample_test.loc[0] = 12647,	0.583333,	0.454545,	0.090909,	1,	0.547270,	0
sample_test_ds = df_to_dataset(sample_test, shuffle=False, batch_size=batch_size)

predictions = model.predict(sample_test_ds)
predictions

       Unnamed: 0     4     5     6  7     8  target
39424           1  0.23  0.29  0.14  1  0.38       1


/home/max/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1699: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


array([[-0.13753884]], dtype=float32)